In [1]:
import datasets as ds

In [2]:
x = ds.concatenate_datasets([
    ds.Dataset.from_parquet("../data/sft-clean/deepscaler.parquet"),
    ds.Dataset.from_parquet("../data/sft-clean/mt-da.parquet"),
    ds.Dataset.from_parquet("../data/sft-clean/mt-eval-train.parquet"),
    ds.Dataset.from_parquet("../data/sft-clean/mt-eval-val.parquet"),
    ds.Dataset.from_parquet("../data/sft-clean/wmt.parquet"),
])

In [3]:
def add_unbounded_rf(example):
    prompt = example["prompt"]
    prompt[0]['content'] = "<reasoning_effort>unbounded</reasoning_effort>\n" + prompt[0]['content']
    return {
        'prompt': prompt,
    }

In [4]:
x = x.map(add_unbounded_rf)

Map:   0%|          | 0/203393 [00:00<?, ? examples/s]

In [5]:
y = ds.concatenate_datasets([
    ds.Dataset.from_parquet("../data/deepscaler.parquet"),
    ds.Dataset.from_parquet("../data/mt-da.parquet"),
    ds.Dataset.from_parquet("../data/mt-eval.parquet"),
    ds.Dataset.from_parquet("../data/wmt24-train.parquet"),
])

In [6]:
def add_disabled_rf(example):
    prompt = example["prompt"]
    prompt[0]['content'] = "<reasoning_effort>disabled</reasoning_effort>\n" + prompt[0]['content']
    return {
        'prompt': prompt,
    }

def extract_gt(example):
    ground_truth = example['reward_model']['ground_truth']
    return {
        'completion': f"<think>\n\n</think>\n<answer>\n{ground_truth}\n</answer>",
    }

y = y.map(add_disabled_rf)
y = y.map(extract_gt)

In [12]:
y_ = y.train_test_split(test_size=0.02)

In [14]:
y_

DatasetDict({
    train: Dataset({
        features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'completion'],
        num_rows: 350076
    })
    test: Dataset({
        features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'completion'],
        num_rows: 7145
    })
})

In [15]:
data = ds.concatenate_datasets(
    [
        x, y_['train']
    ]
)

In [16]:
def merge(example):
    return {
        'multiturn': example['prompt'] + [{"role": "assistant", "content": example['completion']}],
    }

data = data.map(merge)

Map:   0%|          | 0/553469 [00:00<?, ? examples/s]

In [17]:
test = y_['test'].map(merge)

Map:   0%|          | 0/7145 [00:00<?, ? examples/s]

In [18]:
data = data.remove_columns([
   'prompt', 'completion',
])
test = test.remove_columns([
   'prompt', 'completion',
])

In [24]:
data = data.shuffle(seed=42)
test = test.shuffle(seed=42)

data.to_parquet("../data/sft-clean/train.parquet")
test.to_parquet("../data/sft-clean/test.parquet")

Creating parquet from Arrow format:   0%|          | 0/554 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

8833852